# Bolivian Odds Data
## 3. Model Creation

In [2]:
# Importing necessary packages
import pandas as pd
import numpy as np
import matplotlib.style as style
import matplotlib.pyplot as plt
import seaborn as sns
import string
import matplotlib.ticker as ticker
import re
import sklearn
style.use('fivethirtyeight')

# Importing SKLEARN packages
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC


from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [47]:
odds = pd.read_csv('/Users/miketondu/Dropbox/Data Science/Sharpest Minds/odds_data/bolivian_football_odds_prepared.csv',
                  parse_dates= ['date'])

In [48]:
odds=odds.loc['']
odds.head()

,match_href,bookmakers,home_odds,draw_odds,away_odds,payout,date,time,season_year,home_team,away_team,final_result,home_goals,away_goals,first_half_home,first_half_away,second_half_home,second_half_away,total_goals,total_first_half,total_second_half,win_home_or_away,winner,implied_home_odds,implied_draw_odds,implied_away_odds,altitude,upset,home_wins
0,https://www.oddsportal.com/soccer/bolivia/liga...,bwin,1.88,3.35,3.55,0.90,2008-10-19,17:30,2008,Real Potosi,Aurora,1:1,1,1,1,0,0,1,2,1,1,0,Draw,0.532,0.299,0.282,1,0,0.0
1,https://www.oddsportal.com/soccer/bolivia/liga...,bet-at-home,1.85,3.25,3.40,0.88,2008-10-19,17:30,2008,Real Potosi,Aurora,1:1,1,1,1,0,0,1,2,1,1,0,Draw,0.541,0.308,0.294,1,0,0.0
2,https://www.oddsportal.com/soccer/bolivia/liga...,bet-at-home,1.65,3.50,3.95,0.87,2008-10-19,19:30,2008,La Paz,Blooming,2:1,2,1,1,0,1,1,3,1,2,1,La Paz,0.606,0.286,0.253,1,0,0.0
3,https://www.oddsportal.com/soccer/bolivia/liga...,bwin,1.65,3.65,4.30,0.90,2008-10-19,19:30,2008,La Paz,Blooming,2:1,2,1,1,0,1,1,3,1,2,1,La Paz,0.606,0.274,0.233,1,0,0.0
4,https://www.oddsportal.com/soccer/bolivia/liga...,bwin,1.55,3.50,5.50,0.90,2008-10-24,19:30,2008,Blooming,La Paz,5:0,5,0,2,0,3,0,5,2,3,1,Blooming,0.645,0.286,0.182,0,0,0.0


### Base Model
For the purpose of having a base model to compare all other iterations of our model against, we will create a one which will predict a home team win 100% of the time.

In [59]:
# model_df
model_df = odds.copy()
model_df['did_home_team_win'] = np.where(odds.win_home_or_away==1, 1, 0)

# Creating X and Y
X = np.ones(len(odds)).reshape(-1, 1)
y = model_df['did_home_team_win']

# Train/Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
dt = DecisionTreeClassifier()
dt.fit(X_train, y_train)

# Score on test set
print('Accuracy of base model predicting only home wins: ', dt.score(X_test, y_test))

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
                       max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort=False,
                       random_state=None, splitter='best')

Accuracy of base model predicting only home wins:  0.5684289463817023


In [38]:
# random forest
model_df = odds.copy().dropna()
X = model_df.loc[:,odds.columns!='win_home_or_away'] 
X.drop(['date', 'match_href'],axis=1,inplace=True)
X = pd.get_dummies(X)

y = model_df.loc[:,'win_home_or_away']

# Using Skicit-learn to split data into training and testing sets
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.70, random_state=5)


# Import the model we are using
from sklearn.ensemble import RandomForestRegressor
# Instantiate model with 1000 decision trees
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
# Train the model on training data
rf.fit(X_train, y_train);


/Users/miketondu/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3694: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [46]:
# prediction on test set
y_pred=rf.predict(X_test)
rf.score(X_test,y_test)


0.9996268724216552